In [0]:
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.appName('Project').getOrCreate()

In [0]:
df = spark.read.csv('/FileStore/tables/customer_churn.csv',inferSchema=True,header=True)

In [0]:
df.printSchema()

root
-- Names: string (nullable = true)
-- Age: double (nullable = true)
-- Total_Purchase: double (nullable = true)
-- Account_Manager: integer (nullable = true)
-- Years: double (nullable = true)
-- Num_Sites: double (nullable = true)
-- Onboard_date: string (nullable = true)
-- Location: string (nullable = true)
-- Company: string (nullable = true)
-- Churn: integer (nullable = true)

In [0]:
for item in df.head(1)[0]:
  print(item)

Cameron Williams
42.0
11066.8
0
7.22
8.0
2013-08-30 07:00:40
10265 Elizabeth Mission Barkerburgh, AK 89518
Harvey LLC
1

In [0]:
df.summary().show()

+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
summary| Names| Age| Total_Purchase| Account_Manager| Years| Num_Sites| Onboard_date| Location| Company| Churn|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+
 count| 900| 900| 900| 900| 900| 900| 900| 900| 900| 900|
 mean| null|41.81666666666667|10062.82403333334|0.4811111111111111| 5.27315555555555| 8.587777777777777| null| null| null|0.16666666666666666|
 stddev| null|6.127560416916251|2408.644531858096|0.4999208935073339|1.274449013194616|1.7648355920350969| null| null| null| 0.3728852122772358|
 min| Aaron King| 22.0| 100.0| 0| 1.0| 3.0|2006-01-02 04:16:13|00103 Jeffrey Cre...| Abbott-Thompson| 0|
 25%| null| 38.0| 8480.93| 0| 4.45| 7.0| null| null| null| 0|
 50%| null| 42.0| 10041.13| 0| 5.21| 8.0| null| null| null| 0|
 75%| null| 46.0| 11758.69| 1| 6.11| 10.0| null| null| null| 0|
 max|Zachary Walsh| 65.0| 18026.01| 1| 9.15| 14.0|2016-12-28 04:07:38|Unit 9800 Box 287...|Zuniga, Clark and...| 1|
+-------+-------------+-----------------+-----------------+------------------+-----------------+------------------+-------------------+--------------------+--------------------+-------------------+

In [0]:
from pyspark.sql import functions as F

from pyspark.sql.functions import isnan, when, count, col

df.select([count(when(isnan(c), c)).alias(c) for c in df.columns]).show()

+-----+---+--------------+---------------+-----+---------+------------+--------+-------+-----+
Names|Age|Total_Purchase|Account_Manager|Years|Num_Sites|Onboard_date|Location|Company|Churn|
+-----+---+--------------+---------------+-----+---------+------------+--------+-------+-----+
 0| 0| 0| 0| 0| 0| 0| 0| 0| 0|
+-----+---+--------------+---------------+-----+---------+------------+--------+-------+-----+

In [0]:
# from pyspark.sql import functions as F
# from pyspark.sql import Window
# import sys

# w = Window.partitionBy(F.lit(1)).orderBy(F.lit(1)).rowsBetween(-sys.maxsize, 0)

# df.select("*",F.last('INSERT COLUMN NAME HERE',True).over(w).alias('newcol')).show()

In [0]:
df.columns

Out[9]: ['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [0]:
df_staging = (df
              .withColumn('year_Onboard_date',F.year('Onboard_date'))
              .withColumn('month_Onboard_date',F.month('Onboard_date'))
              .withColumn('dayofweek_Onboard_date',F.dayofweek('Onboard_date'))
              .withColumn('dayofmonth_Onboard_date',F.dayofmonth('Onboard_date'))
             )


In [0]:
from pyspark.ml.feature import VectorAssembler, VectorIndexer, OneHotEncoder, StringIndexer

In [0]:
company_indexer = StringIndexer(inputCol='Company',outputCol='Company_index')

In [0]:
company_encoder = OneHotEncoder(inputCol='Company_index',outputCol='Company_vector')

In [0]:
assembler = VectorAssembler(inputCols=['Age'
                                       ,'Total_Purchase'
                                       ,'Account_Manager'
                                       ,'Years'
                                       ,'Num_Sites'
                                       ,'year_Onboard_date'
                                       ,'month_Onboard_date'
                                       ,'dayofweek_Onboard_date'
                                       ,'dayofmonth_Onboard_date'
                                       ,'Company_vector'
                                      ]
                           ,outputCol='features')

In [0]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline

In [0]:
logreg_churn = LogisticRegression(featuresCol='features',labelCol='Churn')

In [0]:
pipeline = Pipeline(stages = [company_indexer
                             ,company_encoder
                             ,assembler
                             ,logreg_churn
                             ])

In [0]:
train_data, test_data = df_staging.randomSplit([0.7, 0.3])

In [0]:
fit_model = pipeline.fit(train_data)

In [0]:
fit_model_analysis = fit_model.stages[-1]

fit_model_summary = fit_model_analysis.summary

fit_model_summary.predictions.describe().show()

+-------+-------------+-----------------+------------------+-------------------+------------------+------------------+-------------------+--------------------+--------------------+------------------+------------------+------------------+----------------------+-----------------------+----------------+----------+
summary| Names| Age| Total_Purchase| Account_Manager| Years| Num_Sites| Onboard_date| Location| Company| Churn| year_Onboard_date|month_Onboard_date|dayofweek_Onboard_date|dayofmonth_Onboard_date| Company_index|prediction|
+-------+-------------+-----------------+------------------+-------------------+------------------+------------------+-------------------+--------------------+--------------------+------------------+------------------+------------------+----------------------+-----------------------+----------------+----------+
 count| 642| 642| 642| 642| 642| 642| 642| 642| 642| 642| 642| 642| 642| 642| 642| 642|
 mean| null|41.87227414330218|10016.602009345792|0.48909657320872274| 5.275062305295941| 8.562305295950155| null| null| null|0.1542056074766355|2010.7445482866044|6.1246105919003115| 4.026479750778816| 16.10436137071651|305.809968847352| 0.0|
 stddev| null| 6.14204284387719| 2453.296207735056| 0.5002708720455912|1.2801855166773426|1.7396934243886604| null| null| null|0.3614273248797815| 3.20444940254699| 3.47938684877732| 1.9376191400083294| 8.778431214272635|184.958910207672| 0.0|
 min| Aaron King| 22.0| 100.0| 0| 1.0| 3.0|2006-01-02 04:16:13|00103 Jeffrey Cre...| Acosta PLC| 0.0| 2006| 1| 1| 1| 0.0| 0.0|
 max|Zachary Walsh| 65.0| 18026.01| 1| 9.15| 14.0|2016-12-07 02:57:32|Unit 8716 Box 686...|Zuniga, Clark and...| 1.0| 2016| 12| 7| 31| 626.0| 0.0|
+-------+-------------+-----------------+------------------+-------------------+------------------+------------------+-------------------+--------------------+--------------------+------------------+------------------+------------------+----------------------+-----------------------+----------------+----------+

In [0]:
predictions_test = fit_model.transform(test_data)

In [0]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [0]:
eval_train = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='Churn')

In [0]:
eval_test = BinaryClassificationEvaluator(rawPredictionCol='prediction',labelCol='Churn')

In [0]:
AUC = eval_test.evaluate(predictions_test)

--------------------------------------------------------------------------- 
 Py4JJavaError Traceback (most recent call last)
 <command-2574753055352988> in <module> 
 ----> 1 AUC = eval_test . evaluate ( predictions_test ) 

 /databricks/spark/python/pyspark/ml/evaluation.py in evaluate (self, dataset, params) 
 70 return self . copy ( params ) . _evaluate ( dataset ) 
 71 else : 
 ---> 72 return self . _evaluate ( dataset ) 
 73 else : 
 74 raise ValueError ( "Params must be a param map but got %s." % type ( params ) ) 

 /databricks/spark/python/pyspark/ml/evaluation.py in _evaluate (self, dataset) 
 100 """
 101 self . _transfer_params_to_java ( ) 
 --> 102 return self . _java_obj . evaluate ( dataset . _jdf ) 
 103 
 104 def isLargerBetter ( self ) : 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1303 answer = self . gateway_client . send_command ( command ) 
 1304 return_value = get_return_value(
 -> 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 
 1307 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 125 def deco ( * a , ** kw ) : 
 126 try : 
 --> 127 return f ( * a , ** kw ) 
 128 except py4j . protocol . Py4JJavaError as e : 
 129 converted = convert_exception ( e . java_exception ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/protocol.py in get_return_value (answer, gateway_client, target_id, name) 
 326 raise Py4JJavaError(
 327 "An error occurred while calling {0}{1}{2}.\n" . 
 --> 328 format(target_id, ".", name), value)
 329 else : 
 330 raise Py4JError(

 Py4JJavaError : An error occurred while calling o995.evaluate.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 5008.0 failed 1 times, most recent failure: Lost task 0.0 in stage 5008.0 (TID 4966, ip-10-172-251-156.us-west-2.compute.internal, executor driver): org.apache.spark.SparkException: Failed to execute user defined function(StringIndexerModel$$Lambda$6070/445752199: (string) => double)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIteratorForCodegenStage1.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anon$1.hasNext(WholeStageCodegenExec.scala:733)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at scala.collection.Iterator$$anon$10.hasNext(Iterator.scala:458)
	at org.apache.spark.util.collection.ExternalSorter.insertAll(ExternalSorter.scala:192)
	at org.apache.spark.shuffle.sort.SortShuffleWriter.write(SortShuffleWriter.scala:62)
	at org.apache.spark.shuffle.ShuffleWriteProcessor.write(ShuffleWriteProcessor.scala:59)
	at org.apache.spark.scheduler.ShuffleMapTask.runTask(ShuffleMapTask.scala:99)
	at org.apache.spark.scheduler.ShuffleMapTask.runTask(ShuffleMapTask.scala:52)
	at org.apache.spark.scheduler.Task.doRunTask(Task.scala:144)
	at org.apache.spark.scheduler.Task.run(Task.scala:117)
	at org.apache.spark.executor.Executor$TaskRunner.$anonfun$run$9(Executor.scala:660)
	at org.apache.spark.util.Utils$.tryWithSafeFinally(Utils.scala:1581)
	at org.apache.spark.executor.Executor$TaskRunner.run(Executor.scala:663)
	at java.util.concurrent.ThreadPoolExecutor.runWorker(ThreadPoolExecutor.java:1149)
	at java.util.concurrent.ThreadPoolExecutor$Worker.run(ThreadPoolExecutor.java:624)
	at java.lang.Thread.run(Thread.java:748)
Caused by: org.apache.spark.SparkException: Unseen label: Little, White and Davis. To handle unseen labels, set Param handleInvalid to keep.
	at org.apache.spark.ml.feature.StringIndexerModel.$anonfun$getIndexer$1(StringIndexer.scala:406)
	at org.apache.spark.ml.feature.StringIndexerModel.$anonfun$getIndexer$1$adapted(StringIndexer.scala:391)
	... 20 more

Driver sta

In [0]:
AUC

In [0]:
logreg_model_final = pipeline.fit(df_staging)

In [0]:
new_customers = spark.read.csv('/FileStore/tables/new_customers.csv')

In [0]:
new_customers.printSchema()

In [0]:
new_customers_model = logreg_model_final.transform(new_customers)

In [0]:
new_customers_model.select('Company','prediction').show()